## 필수과제 2

* 리뷰데이터를 전처리하는 코드 ( 정규표현식, 불용어 등으로 처리하여 리뷰데이터 정리 후 )
* 해당 전처리는 다 코드랑, 내용 정리해서 어떤식으로 진행했는지 공유해 주세요!
* 태깅해서 정리하여 코사인 유사도를 공유해 주세요!

## 필수과제 2-1

* 전처리하지 않고 베이스로 먼저했던 유사도 리뷰

In [ ]:
# 라이브러리 불러오기
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# 데이터 불러오기
df = pd.read_csv('movie_rv.csv')

# 결측치 제거
df_sp = df.dropna()

# 텍스트 데이터 추출
df_docu = df_sp['document']
df_docu

0                                       아 더빙.. 진짜 짜증나네요 목소리
1                         흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                         너무재밓었다그래서보는것을추천한다
3                             교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4         사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
                                ...                        
149995                                  인간이 문제지.. 소는 뭔죄인가..
149996                                        평점이 너무 낮아서...
149997                      이게 뭐요? 한국인은 거들먹거리고 필리핀 혼혈은 착하다?
149998                          청춘 영화의 최고봉.방황과 우울했던 날들의 자화상
149999                             한국 영화 최초로 수간하는 내용이 담긴 영화
Name: document, Length: 149995, dtype: object

In [ ]:
# CountVectorizer 객체 생성
re_cv = CountVectorizer(max_features=5000)

# 텍스트의 벡터화 적용
re_cnv = re_cv.fit_transform(df_docu)

# 기준점 리뷰
r = df_docu.loc[4056]
r

'따뜻하고 좋은영화 였어요. 근데 인연이란 참 미요한거네요.'

In [ ]:
# 텍스트의 벡터화 적용
source_rv1 = re_cv.transform([r])

# 코사인 유사도 계산
sim_res1 = cosine_similarity(source_rv1, re_cnv)

# 기준점 리뷰와의 상위 유사도 10개 추출
df_docu.loc[(-sim_res1[0]).argsort()[:10]]

4056                       따뜻하고 좋은영화 였어요. 근데 인연이란 참 미요한거네요.
58320                               마지막 장면서 화가애인두 죽은걸루 보이는대
12201                  이종석 코믹연기 진짜웃김 근데 스토리상 개암걸림 박보영보 완전이쁨
112600              원작의 어설픈 변형과 미스 캐스팅..결과는 제작비의 절반만 벌어들임..
57363                                                 억지스럽다
16526                     별로네여 근데 대성생각보다 잘햇음 연예인더빙 안좋아하는데..
83967                                      요즘 귀신은 랜선을 타고~ ~
45253     장국영의 황홀한 연기와 날이 선 왕가위의 완벽한 조합. 홍콩에서 왜 늘 위대한 영화...
69595                   멜깁슨을 위한 영화. 거칠지만 유쾌한 버디 형사물의 뉴 웨이브.
71388                                      너무 아동틱한 영화가 된듯..
Name: document, dtype: object

-> 기준점의 리뷰는 긍정적인데, 부정적인 표현의 리뷰가 많이 보인다. 코사인 유사도가 정확하지 않다고 해석할 수 있고, 전처리가 필수적으로 되어야 한다고 생각한다.

## 필수과제 2-2

* 전처리 후의 베이스로 했던 유사도와의 비교

### 정규표현식


In [ ]:
# 정규표현식을 통해 초성(ㅋㅋㅋ,ㅠㅠㅠ), 영어, 이모티콘 제거하기

import re

def RE(review) :
  rev = re.sub('[^가-힣\\s]', " ", review) # 한글과 공백만 남기기
  return rev

# df_docu에 적용
re_rev = []
for i in df_docu :
  re_rev.append(RE(i))

re_rev[:10]

['아 더빙   진짜 짜증나네요 목소리',
 '흠   포스터보고 초딩영화줄    오버연기조차 가볍지 않구나',
 '너무재밓었다그래서보는것을추천한다',
 '교도소 이야기구먼   솔직히 재미는 없다  평점 조정',
 '사이몬페그의 익살스런 연기가 돋보였던 영화 스파이더맨에서 늙어보이기만 했던 커스틴 던스트가 너무나도 이뻐보였다',
 '막 걸음마 뗀  세부터 초등학교  학년생인  살용영화       별반개도 아까움 ',
 '원작의 긴장감을 제대로 살려내지못했다 ',
 '별 반개도 아깝다 욕나온다 이응경 길용우 연기생활이몇년인지  정말 발로해도 그것보단 낫겟다 납치 감금만반복반복  이드라마는 가족도없다 연기못하는사람만모엿네',
 '액션이 없는데도 재미 있는 몇안되는 영화',
 '왜케 평점이 낮은건데  꽤 볼만한데   헐리우드식 화려함에만 너무 길들여져 있나 ']

### 형태소 분리

In [ ]:
# konlpy 패키지 설치
! pip install konlpy

In [ ]:
# konlpy 패키지 불러오기
from konlpy.tag import Okt

# Okt 객체 생성
tw_tag = Okt()

# 토크나이저 함수 생성
def my_token(doc):
    return [token for token, pos in tw_tag.pos(doc) if pos in ['Noun', 'Verb', 'Adjective', 'Adverb']]

# 하이퍼파라미터에 함수 활용
re_cv_mt = CountVectorizer(max_features=5000, tokenizer= my_token)

# 텍스트의 벡터화 적용
re_cnv_mt=re_cv_mt.fit_transform(re_rev)

# 내가 선정한 피처들은 무엇이 있는지?
re_cv_mt.get_feature_names_out()[:100]

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array(['가', '가끔', '가는', '가서', '가슴', '가장', '가족', '가지', '가치', '각본', '간다',
       '간만', '갈수록', '감', '감독', '감동', '감사합니다', '감상', '감성', '감정', '갑자기',
       '강', '강추', '같네요', '같다', '같습니다', '같아', '같아서', '같아요', '같은', '같은데',
       '같음', '같이', '개', '개그', '개봉', '개연', '개인', '걍', '거', '거기', '거리',
       '거의', '거지', '건', '건가', '건지', '걸', '걸작', '것', '게', '게임', '결국', '결론',
       '결말', '결혼', '경찰', '계속', '고', '곳', '공감', '공포', '공포영화', '과', '과거',
       '과정', '관', '관객', '관계', '관람', '관심', '광고', '괜찮은', '괜히', '굉장히', '교훈',
       '구', '구성', '군대', '굳', '굿', '귀신', '그', '그것', '그게', '그나마', '그냥',
       '그녀', '그닥', '그대로', '그때', '그래', '그래도', '그래서', '그래픽', '그런', '그런지',
       '그렇게', '그렇고', '그리'], dtype=object)

### 불용어 처리하기

In [ ]:
# 불용어 리스트 생성하기 위한 형태소 개수 파악
words = re_cv_mt.get_feature_names_out()
count = re_cnv_mt.sum(axis=0).tolist()[0]

# 데이터프레임 생성
df_count = pd.DataFrame({'Word': words, 'Count': count})

# 가장 많이 쓰인 형태소 30개 추출
df_count_sorted = df_count.sort_values(by='Count', ascending=False)
df_count_sorted.head(50)

,Word,Count
618,영화,50808
157,너무,11138
783,정말,9647
847,진짜,8345
670,이,8003
775,점,7932
605,연기,6434
915,평점,6325
49,것,6319
877,최고,6038


In [ ]:
stop_words = ['이', '점', '것', '다', '하는', '때', '내', '거', '뭐', '그', '좀', '이런', '할', '이건' ]

# 하이퍼파라미터에 함수 활용
re_cv_sw = CountVectorizer(max_features=5000, tokenizer= my_token, stop_words= stop_words)

# 텍스트의 벡터화 적용
re_cnv_sw=re_cv_sw.fit_transform(re_rev)

# 내가 선정한 피처들은 무엇이 있는지?
re_cv_sw.get_feature_names_out()[:100]

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


array(['가', '가끔', '가는', '가서', '가슴', '가장', '가족', '가지', '가치', '각본', '간다',
       '간만', '갈수록', '감', '감독', '감동', '감사합니다', '감상', '감성', '감정', '갑자기',
       '강', '강추', '같네요', '같다', '같습니다', '같아', '같아서', '같아요', '같은', '같은데',
       '같음', '같이', '개', '개그', '개봉', '개연', '개인', '걍', '거기', '거리', '거의',
       '거지', '건', '건가', '건지', '걸', '걸작', '게', '게임', '결국', '결론', '결말',
       '결혼', '경찰', '계속', '고', '고민', '고생', '곳', '공감', '공포', '공포영화', '과',
       '과거', '과정', '관', '관객', '관계', '관람', '관심', '광고', '괜찮은', '괜히', '굉장히',
       '교훈', '구', '구성', '군대', '굳', '굿', '귀신', '그것', '그게', '그나마', '그냥',
       '그녀', '그닥', '그대로', '그때', '그래', '그래도', '그래서', '그래픽', '그런', '그런지',
       '그렇게', '그렇고', '그리', '그림'], dtype=object)

* 전처리 후 코사인 유사도 비교

In [ ]:
# 기준점 리뷰
r = df_docu.loc[4056]
r

'따뜻하고 좋은영화 였어요. 근데 인연이란 참 미요한거네요.'

In [ ]:
# 텍스트의 벡터화 적용
source_rv2 = re_cv_sw.transform([r])

# 코사인 유사도 계산
sim_res2 = cosine_similarity(source_rv2, re_cnv_sw)

# 기준점 리뷰와의 상위 유사도 10개 추출
df_docu.loc[(-sim_res2[0]).argsort()[:10]]

4056             따뜻하고 좋은영화 였어요. 근데 인연이란 참 미요한거네요.
85179                                     고잉홈 슬펐다
60148            심각하게 재밌습니다 삼십분 드라마라는게 원망스러울정도입니다
113567        새로 나올수록 내리막길 걷는 주온시리즈입니다..비디오판 강추..
62436                                       총체적난국
109832               대학1학년때 얼마나 재밋게 반는지~~~책도보고 ㅋㅋ
48637                    재미없어 쓰레기.. 2점부터 점수준것들 M창
52809                              뭐지 결국엔 성형이 답이네
139708               언제 봐도 재미있는 영화 참 보면서 뭔가 느껴지네요
149217    샤를로뜨 갱스부르 팬이라면 볼만한 영화, OST가 웃음 포인트일줄이야.
Name: document, dtype: object

-> 전처리 전보다 부정적인 표현의 리뷰가 줄어들고, 긍정적인 표현의 리뷰가 늘어났다. 전처리 후 유사한 변수를 보다 더 잘 찾아낸 것으로 해석할 수 있다.